## Import

In [ ]:
import sys, os
path2add = os.path.normpath(os.path.abspath(os.path.join(os.path.curdir, os.path.pardir)))
print(path2add)

if (not (path2add in sys.path)) :
    sys.path.append(path2add)

In [ ]:
from scipy.stats import mode
from pathlib import Path

import BeamTestHelpers as helper
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mplhep as hep
hep.style.use('CMS')

import yaml

## Specify board information

In [ ]:
with open("../board_configs_yaml/CERN_Irrad_2025.yaml", "r") as file:
    fig_configs = yaml.safe_load(file)

print(fig_configs.keys())

given_run = 'run0'
selected_fig_config = fig_configs[given_run]

for id in selected_fig_config.keys():
    selected_fig_config[id]['title'] = f"{selected_fig_config[id]['name']} HV{selected_fig_config[id]['HV']}V OS:{selected_fig_config[id]['offset']}"

selected_fig_config

## Set output directory depending on TB campaign

In [ ]:
# current_dir = Path('./')
# output_mother_dir = current_dir / 'etroc_TB_figs'
# output_mother_dir.mkdir(exist_ok=True)

# ### Now you need change the directory name per campaign
# ### Naming rule is this:
# ### <TB location>_TB_MonthYear
# ### E.g. desy_TB_Apr2024, cern_TB_Sep2023, fnal_TB_Jul2024

# output_campaign_dir = output_mother_dir / 'desy_TB_Dec2024'
# # output_campaign_dir.mkdir(exist_ok=True)

## Read input files

In [ ]:
from natsort import natsorted
files = Path('.').glob('./loop_0.feather')
files = natsorted(list(files))
files

In [ ]:
dataframes = []
columns_to_read = ['evt', 'ea', 'board', 'row', 'col', 'toa', 'tot', 'cal']

for idx, ifile in enumerate(files):
    tmp_df = pd.read_feather(ifile, columns=columns_to_read)
    nevt = tmp_df['evt'].nunique()
    if idx > 0:
        tmp_df['evt'] += nevt
    dataframes.append(tmp_df)
    del tmp_df

df = pd.concat(dataframes)
del dataframes

In [ ]:
helper.plot_TOA_correlation_hit(df, 0, 1, 'irrad', selected_fig_config)

## Make occupancy map

In [ ]:
helper.plot_occupany_map(df, board_info=selected_fig_config, tb_loc='irrad', extra_cms_title='ETL ETROC IRRad')

In [ ]:
h_inclusive = helper.return_hist(input_df=df, board_info=selected_fig_config, hist_bins=[100, 128, 128])
chip_figtitles = [val['title'] for _, val in selected_fig_config.items()]
helper.plot_1d_TDC_histograms(input_hist=h_inclusive, tb_loc='irrad', extra_cms_title='ETL ETROC IRRad',
                                fig_tag=chip_figtitles, slide_friendly=True)

## CAL code filtering

In [ ]:
### CAL code filtering
cal_table = df.pivot_table(index=["row", "col"], columns=["board"], values=["cal"], aggfunc=lambda x: x.mode().iat[0])
cal_table = cal_table.reset_index().set_index([('row', ''), ('col', '')]).stack().reset_index()
cal_table.columns = ['row', 'col', 'board', 'cal_mode']

merged_df = pd.merge(df[['board', 'row', 'col', 'cal']], cal_table, on=['board', 'row', 'col'])
merged_df['board'] = merged_df['board'].astype('uint8')
merged_df['cal_mode'] = merged_df['cal_mode'].astype('int16')
cal_condition = abs(merged_df['cal'] - merged_df['cal_mode']) <= 3
del cal_table, merged_df
cal_filtered_df = df[cal_condition].reset_index(drop=True)
del df, cal_condition

In [ ]:
h_inclusive = helper.return_hist(input_df=cal_filtered_df, board_info=selected_fig_config, hist_bins=[100, 128, 128])
chip_figtitles = [val['title'] for _, val in selected_fig_config.items()]
helper.plot_1d_TDC_histograms(input_hist=h_inclusive, tb_loc='irrad', extra_cms_title='ETL ETROC IRRad',
                                fig_tag=chip_figtitles, slide_friendly=True)

## Make spatial correlation plots

In [ ]:
import itertools
from matplotlib.ticker import NullLocator

df = cal_filtered_df

# --- Assume your full DataFrame is 'df' ---

# 1. Get all unique board IDs and create all unique pairs
board_ids = np.sort(df['board'].unique())
board_pairs = list(itertools.combinations(board_ids, 2))

# --- Define plotting settings ---
# Bins and range for 16x16 pixels
plot_bins = 16
plot_range = [[-0.5, 15.5], [-0.5, 15.5]]
plot_ticks = np.arange(16) # [0, 1, ..., 15]

# ===============================================
#  1. PLOT ROW CORRELATIONS (2x3 Grid)
# ===============================================
print("Generating Row Correlation Plots...")
# Create a 2x3 grid of plots.
fig_row, axes_row = plt.subplots(2, 3, figsize=(24, 15))
# Flatten the 2x3 array of axes into a 1D array for easy iteration
axes_row = axes_row.flatten()

# Zip pairs and axes to plot on each
for (b_i, b_j), ax in zip(board_pairs, axes_row):

    # 1. Separate data for the pair
    df_bi = df[df['board'] == b_i][['evt', 'row', 'col']]
    df_bj = df[df['board'] == b_j][['evt', 'row', 'col']]

    # 2. Merge on 'evt'
    # Use f-strings to create dynamic suffixes
    suffix_i = f'_b{b_i}'
    suffix_j = f'_b{b_j}'
    df_corr = pd.merge(df_bi, df_bj, on='evt', suffixes=(suffix_i, suffix_j))

    # Define the column names we just created
    row_i_col = f'row{suffix_i}' # e.g., 'row_b0'
    row_j_col = f'row{suffix_j}' # e.g., 'row_b1'

    # 3. Plot on the specific axis 'ax'
    if not df_corr.empty:
        h = ax.hist2d(
            df_corr[row_i_col],
            df_corr[row_j_col],
            bins=plot_bins,
            range=plot_range,
            cmin=1
        )
        fig_row.colorbar(h[3], ax=ax, label="Hit Pairs") # Add colorbar to this subplot

    # 4. Set labels and ticks
    ax.set_xlabel(f"Row (Board {selected_fig_config[b_i]['short']})", fontsize=20)
    ax.set_ylabel(f"Row (Board {selected_fig_config[b_j]['short']})", fontsize=20)

    ax.set_xticks(plot_ticks)
    ax.set_yticks(plot_ticks)
    ax.xaxis.set_minor_locator(NullLocator())
    ax.yaxis.set_minor_locator(NullLocator())
    ax.tick_params(axis='both', which='major', labelsize=17)

# Clean up layout
fig_row.tight_layout()

# ===============================================
#  2. PLOT COLUMN CORRELATIONS (2x3 Grid)
# ===============================================
print("Generating Column Correlation Plots...")
fig_col, axes_col = plt.subplots(2, 3, figsize=(24, 15))
axes_col = axes_col.flatten()

for (b_i, b_j), ax in zip(board_pairs, axes_col):

    # 1. Separate (This step is redundant if memory is a concern,
    # but clear for demonstration)
    df_bi = df[df['board'] == b_i][['evt', 'row', 'col']]
    df_bj = df[df['board'] == b_j][['evt', 'row', 'col']]

    # 2. Merge
    suffix_i = f'_b{b_i}'
    suffix_j = f'_b{b_j}'
    df_corr = pd.merge(df_bi, df_bj, on='evt', suffixes=(suffix_i, suffix_j))

    # Define the *column* names
    col_i_col = f'col{suffix_i}' # e.g., 'col_b0'
    col_j_col = f'col{suffix_j}' # e.g., 'col_b1'

    # 3. Plot on 'ax'
    if not df_corr.empty:
        h = ax.hist2d(
            df_corr[col_i_col],
            df_corr[col_j_col],
            bins=plot_bins,
            range=plot_range,
            cmin=1
        )
        fig_col.colorbar(h[3], ax=ax, label="Hit Pairs")

    # 4. Set labels and ticks
    ax.set_xlabel(f"Column (Board {selected_fig_config[b_i]['short']})", fontsize=20)
    ax.set_ylabel(f"Column (Board {selected_fig_config[b_j]['short']})", fontsize=20)

    ax.set_xticks(plot_ticks)
    ax.set_yticks(plot_ticks)
    ax.xaxis.set_minor_locator(NullLocator())
    ax.yaxis.set_minor_locator(NullLocator())
    ax.tick_params(axis='both', which='major', labelsize=17)

# Clean up layout
fig_col.tight_layout()
